In [2]:
import pandas as pd
import numpy as np
import openpyxl

In [3]:
data = pd.read_excel('Data.xlsx', engine = "openpyxl")
data.head()

,Supply Site Code,SKU,Location Code,Location Type,MinDOC (Hl),Reorder Point (Hl),MaxDOC (Hl),Closing Stock,Distributor Orders,Current CS/MIN,Current CS/ROP,Current CS/MAX,Available to Deploy,Scenario
0,PL-1505,85021,PL-1505,DEP,19218.603508,70565.619828,157453.679828,0.00,0,0.000000,0.000000,0.000000,16372.22,4
1,PL-1505,85023,PL-1505,DEP,3228.341505,40347.904084,54904.264084,0.00,0,0.000000,0.000000,0.000000,24081.27,3
2,PL-1505,85023,PL-1609,DEP,82.098601,205.696898,439.216898,624.33,0,7.604636,3.035194,1.421462,24081.27,3
3,PL-1505,85023,PL-1831,DEP,139.463970,532.329491,1231.769491,0.00,0,0.000000,0.000000,0.000000,24081.27,3
4,PL-1505,85023,PL-1832,DEP,167.589483,693.650331,1617.650331,4184.00,0,24.965767,6.031858,2.586467,24081.27,3


In [27]:
data.shape
data.dtypes
data.columns

Index(['Supply Site Code', 'SKU', 'Location Code', 'Location Type',
       'MinDOC (Hl)', 'Reorder Point (Hl)', 'MaxDOC (Hl)', 'Closing Stock',
       'Distributor Orders', 'Current CS/MIN', 'Current CS/ROP',
       'Current CS/MAX', 'Available to Deploy', 'Scenario'],
      dtype='object')

In [5]:
clean_data = data[(data['Available to Deploy'] != 0)&(data['Reorder Point (Hl)'] != 0)]
clean_data.shape

(1964, 14)

In [6]:
clean_data.head()

,Supply Site Code,SKU,Location Code,Location Type,MinDOC (Hl),Reorder Point (Hl),MaxDOC (Hl),Closing Stock,Distributor Orders,Current CS/MIN,Current CS/ROP,Current CS/MAX,Available to Deploy,Scenario
0,PL-1505,85021,PL-1505,DEP,19218.603508,70565.619828,157453.679828,0.00,0,0.000000,0.000000,0.000000,16372.22,4
1,PL-1505,85023,PL-1505,DEP,3228.341505,40347.904084,54904.264084,0.00,0,0.000000,0.000000,0.000000,24081.27,3
2,PL-1505,85023,PL-1609,DEP,82.098601,205.696898,439.216898,624.33,0,7.604636,3.035194,1.421462,24081.27,3
3,PL-1505,85023,PL-1831,DEP,139.463970,532.329491,1231.769491,0.00,0,0.000000,0.000000,0.000000,24081.27,3
4,PL-1505,85023,PL-1832,DEP,167.589483,693.650331,1617.650331,4184.00,0,24.965767,6.031858,2.586467,24081.27,3


In [8]:
breweries = clean_data['Supply Site Code'].unique()

In [49]:
def scen4(avbl_to_dply, grid):
    new_grid = pd.DataFrame(columns = grid.columns)
    while avbl_to_dply>0:
        min_ratio = grid['Current CS/ROP'].min()
        new_grid = new_grid.append(grid[(grid['Current CS/ROP']== min_ratio)])
        grid = grid[(grid['Current CS/ROP']!= min_ratio)]
        next_min = grid['Current CS/ROP'].min()
        sum_ROP = new_grid['Reorder Point (Hl)'].sum()
        sum_step = (next_min-min_ratio)*(sum_ROP)
        if avbl_to_dply>=sum_step:
            avbl_to_dply-= sum_step
            new_grid['Current CS/ROP'] = next_min
        else:
            new_grid['Current CS/ROP'] = new_grid['Current CS/ROP']+ (avbl_to_dply/sum_ROP)
            avbl_to_dply = 0
    print(next_min)
    new_grid = new_grid.append(grid)
    new_grid.sort_values('Current CS/ROP', inplace =True)
    print (new_grid[['Supply Site Code','SKU','Location Code','Current CS/ROP','Available to Deploy']].head(20))


In [50]:
for site in breweries:
    brewery = clean_data[(clean_data['Supply Site Code']==site)]
    products = brewery['SKU'].unique()
    for product in products:
        grid = brewery[(brewery['SKU']==product)]
        avbl_to_dply = grid['Available to Deploy'].mean()
        scenario = grid['Scenario'].mean()
        if scenario == 4:
            print (avbl_to_dply, scenario)
            print (grid[['Supply Site Code','SKU','Location Code','Current CS/ROP','Available to Deploy']].head(20))
            scen4(avbl_to_dply, grid)
            #print (new_grid[['Supply Site Code','SKU','Location Code','Current CS/ROP','Available to Deploy']].head(20))
    break

16372.22 4.0
  Supply Site Code    SKU Location Code  Current CS/ROP  Available to Deploy
0          PL-1505  85021       PL-1505             0.0             16372.22
nan
  Supply Site Code    SKU Location Code  Current CS/ROP  Available to Deploy
0          PL-1505  85021       PL-1505        0.232014             16372.22
1149.35 4.0
    Supply Site Code    SKU Location Code  Current CS/ROP  Available to Deploy
150          PL-1505  86631       PL-1501        3.366275              1149.35
151          PL-1505  86631       PL-1502        4.631350              1149.35
152          PL-1505  86631       PL-1503        1.117632              1149.35
153          PL-1505  86631       PL-1504        0.735709              1149.35
154          PL-1505  86631       PL-1505        0.000000              1149.35
155          PL-1505  86631       PL-1506        1.867709              1149.35
156          PL-1505  86631       PL-1607        3.428949              1149.35
157          PL-1505  86631    